In [1]:
!pip install opencv-python kociemba

In [2]:
!pip install numpy

In [3]:
pip install kociemba


Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install RubikTwoPhase

In [2]:
!pip install pyserial

   ---------------------------------------- 0.0/90.6 kB ? eta -:--:--
   ------------------ --------------------- 41.0/90.6 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 90.6/90.6 kB 1.3 MB/s eta 0:00:00


In [5]:
import cv2
import numpy as np
#import kociemba
import twophase.solver  as sv
import serial
import time
import serial.tools.list_ports

In [ ]:
# list of serial ports available

import serial.tools.list_ports
ports = serial.tools.list_ports.comports()

for p in ports:
    print(p)

In [ ]:
"""
Methodlogy:

1) 9 circles in the middle of webcam which will capture colors of cube
2) Take those circles, make it into hsv colorspace
3) 
3) 3d array will be used as input, just convert it to string using join() method

for when we have to look at color, we can hardcode logic that identifies each new face
based on the color of center piece, and then appends it accordingly to into 3d array

For Kociemba Module

U: Yellow Center Face
L: Blue Center Face
F: Red Center Face
R: Green Center Face
B: Blue Center Face
D: White Center Face
"""

In [1]:
import cv2
import numpy as np

# sets video capture to default webcam
# then we set the dimensions of the video capture

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)


# define circle positions
center_coordinate_x = 640
center_coordinate_y = 360  # Fixed typo here
offset_pixel_distance = 60


# calculated circle positions organized as a grid
circle_coordinates = [(center_coordinate_x - offset_pixel_distance, center_coordinate_y + offset_pixel_distance), (center_coordinate_x, center_coordinate_y + offset_pixel_distance), (center_coordinate_x + offset_pixel_distance, center_coordinate_y + offset_pixel_distance), # top row
                      (center_coordinate_x - offset_pixel_distance, center_coordinate_y), (center_coordinate_x, center_coordinate_y), (center_coordinate_x + offset_pixel_distance, center_coordinate_y), # middle row
                      (center_coordinate_x - offset_pixel_distance, center_coordinate_y - offset_pixel_distance), (center_coordinate_x, center_coordinate_y - offset_pixel_distance), (center_coordinate_x + offset_pixel_distance, center_coordinate_y - offset_pixel_distance)] # bottom row

# 3d array that holds the input cube state
input_cube_array = np.empty((6,3,3), dtype = str)

#compare hsv values and return color
def get_color(hue, sat, val):
    if sat < 50 and val > 180:
        return "W"
    elif (hue < 10 or hue > 170) and sat > 70:  # Adjusted red range and added saturation check
        return "R"
    elif 10 <= hue < 25 and sat > 70 and val > 50:  # Adjusted orange range and added saturation/value checks
        return "O"
    elif 25 <= hue < 35 and sat > 50:
        return "Y"
    elif 35 <= hue < 85 and sat > 40:
        return "G"
    elif 85 <= hue < 130 and sat > 40:
        return "B"
    else:
        return "U"  # unknown
    
def array_to_string(input_cube_array):

    twophase_output_string = ""
    
    DIMENSION_1 = 6
    DIMENSION_2 = 3
    DIMENSION_3 = 3

    for x in range(DIMENSION_1):
        for y in range(DIMENSION_2):
            for z in range(DIMENSION_3):
                twophase_output_string += input_cube_array[x][y][z]
    
    return twophase_output_string

# order of faces to scan
face_order = ["Yellow", "Green", "Red", "White", "Blue", "Orange"]
face_index = 0

# webcam logic
while True:

    # something new i learned
    # ret is a bool variable that returns true if the frame is available
    # frame is an image array vector captured based on default frames per second
    
    ret, frame = cap.read()

    # if webcame is not working
    if not ret:
        print("Failed to grab frame.")
        break

    # cv2.cv.flip(src, flipCode[, dst] )
    # 1 for dst which will mirror live stream

    frame = cv2.flip(frame, 1)

    # converting frame to hsv
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)


    # now add the circles onto the frame
    # we need to loop through the coordinates and draw the circles
    # for j,k in my_list (stack overflow)
    for (c_x, c_y) in circle_coordinates:

        cv2.circle(frame, (c_x, c_y), 7, (25, 25, 25), 2)
        pixel_center_hsv = hsv_frame[c_y, c_x]  # Changed from frame to hsv_frame
        color = get_color(pixel_center_hsv[0], pixel_center_hsv[1], pixel_center_hsv[2])
        
        # display the current color to the right
        cv2.putText(frame, color, (c_x + 10, c_y), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0, 0, 0),2)

    # display current being displayed
    cv2.putText(frame, f"Scanning: {face_order[face_index]} Center Face", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Rubik's Cube Scanner", frame)


    key = cv2.waitKey(1) & 0xFF

    # logic for taking input for cube

    # if spacebar is pressed, capture the colors
    if key == ord(' '):
        for i, (c_x, c_y) in enumerate(circle_coordinates):
            print(f"Scanning face: {face_order[face_index]}")
            # Images in OpenCV are represented by NumPy arrays. To access a particular image pixel, all we need to do is pass in the (x, y)-coordinates as image[y, x]:
            pixel_center_hsv = hsv_frame[c_y,c_x]  #
            
            color = get_color(pixel_center_hsv[0], pixel_center_hsv[1], pixel_center_hsv[2])

            input_cube_array[face_index, i // 3, i % 3] = color
            
            print(f"Square {i+1}: {color}")

        face_index +=1
    
        # if all 6 sides are scanned, we are done
        if (face_index >=6):
            print("All faces scanned.")
            break


    # have a stop condition (click q for quit)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#print("Final Cube State: ")
#print(input_cube_array)

def color_notation_to_kociemba_notation(str):

    if str == "Y":
        return "U"
    
    elif str == "B":
        return "L"
    
    elif str == "R":
        return "F"
    
    elif str == "G":
        return "R"
    
    elif str == "O":
        return "B"
    
    elif str == "W":
        return "D"
    
    else:
        return "?"


def array_to_string(input_cube_array):

    twophase_output_string = ""
    
    DIMENSION_1 = 6
    DIMENSION_2 = 3
    DIMENSION_3 = 3

    for x in range(DIMENSION_1):
        for y in range(DIMENSION_2):
            for z in range(DIMENSION_3):
                # we need to convert from colors to the actual sides
                str_temp = ""
                str_temp = color_notation_to_kociemba_notation(input_cube_array[x][y][z])
                twophase_output_string += str_temp
    
    return twophase_output_string

def two_phase_solution_to_arduino(input_string):

    split_string = input_string.split()
    arduino_commands = []

    for move in split_string:
        if move == '(':
            continue

        face = move[0]
        quantity_of_move = move[1]



        if quantity_of_move == '1':
                arduino_commands.append(face)
        elif quantity_of_move == '2':
                arduino_commands.extend([face], [face])
        elif quantity_of_move == '3':
            for i in range(3):
                arduino_commands.extend([face], [face], [face])

    return arduino_commands

cube_string = array_to_string(input_cube_array)
print(f"Cube Input String: {cube_string}")

try: 
    solution = sv.solve(cube_string, 25, 5)
    print(f"Solution: {solution}")

except Exception as e:
    print(f"Error: {e}")


# this is the array 
output_array = two_phase_solution_to_arduino(solution)

ser = serial.Serial('/dev/ttyACM0', 9600, timeout=5)

def send_move(move): # this will be called within the 
    ser.write(move.encode())
    response = ser.readline().decode().strip()
    if response != "OK":
        raise Exception(f"Move {move} failed")
    
def send_moves(moves):
    for move in moves:
        send_move(move)
        print(f"Completed move: {move}")

# try except statement for send moves method

try: 
    send_moves(output_array)
    print("All moves completed succesfully.")

except Exception as e:
    print(f"Error ")

    



Failed to grab frame.
Cube Input String: ??????????????????????????????????????????????????????
Error: name 'sv' is not defined


NameError: name 'solution' is not defined

In [ ]:
class Motor:
    def __init__(self, name):
        self.name = name

    def move(self, direction):
        